# Неперсонализованные рекомендации (Non-Personalised Recommendations)

Когда пользователь только приходит на сервис мы ничего не знаем о его вкусах, так как откликов от него еще не было. В таком случае применяют неперсонализованные рекоммендации. (, рекомендации основанные не на истории )

### Популярные рекоммендации

Первая мысль которая может придти какой товар порекомендовать - это порекомендовать популярный товар. Но тогда встает вопрос - что значит популярный?
Это может значить например:
<ul>
    <li>наиболее просматриваемый</li>
    <li>наиболее кликабельный</li>
    <li>наиболее покупаемый</li>
    <li>товары которые берут чаще всего с текущими в вашей корзине покупок</li>
</ul>
Выбор определения зависит от конкретного сервиса и каких целей вы приследуете 

In [1]:
import pandas as pd

In [29]:
df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv', sep=',')

In [39]:
df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

In [30]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [45]:
min_date = df_ratings['timestamp'].max().normalize() - pd.DateOffset(days=30)

In [47]:
df_ratings['timestamp'].max()

Timestamp('2018-09-24 14:27:30')

In [62]:
df_ratings.loc[df_ratings['timestamp'] > min_date, 'movieId'].value_counts()

122906    4
58559     4
122912    4
187593    4
79132     4
         ..
106873    1
39292     1
175485    1
4014      1
32799     1
Name: movieId, Length: 712, dtype: int64

In [46]:
min_date

Timestamp('2018-08-25 00:00:00')

In [70]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit='s').sort_values()

In [ ]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [87]:
recommender = PopularRecommender(days=21, item_column='movieId', dt_column='timestamp')

In [88]:
recommender.fit(df_ratings)

In [89]:
recommender.recommend()

array([122906, 187593,  91529, 122912, 122916,   1089, 119145, 112552,
       177765, 164179])

In [90]:
pd.DataFrame(recommender.recommend(), columns=['movieId']).merge(df_movies)

,movieId,title,genres
0,122906,Black Panther (2017),Action|Adventure|Sci-Fi
1,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi
2,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX
3,122912,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi
4,122916,Thor: Ragnarok (2017),Action|Adventure|Sci-Fi
5,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
6,119145,Kingsman: The Secret Service (2015),Action|Adventure|Comedy|Crime
7,112552,Whiplash (2014),Drama
8,177765,Coco (2017),Adventure|Animation|Children
9,164179,Arrival (2016),Sci-Fi


## Демографические группы, сегменты популярных рекоммендаций 

Однако легко может случится так, что популярные продукты не соответствуют конкретно вашим вкусам. Но вы можете принадлежать некоторой демографической группе, что может хорошо определить ваши вкусы.

Если в начале у нас имеется информация о самом пользователе, такие как:
<ul>
    <li>Пол</li>
    <li>Возраст</li>
    <li>Семейный статус</li>
    <li>Социальный статус</li>
    <li>Вид и область занятости</li>
    <li>Местоположение</li>
    <li>итд</li>
</ul>
Можно провести аналитики на отдельных группах и подобрать более специфичную

In [ ]:
Assosiative 

### Всегда ли полезны рекоммендации?

Нет, не всегда. Если посмотреть 